In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.pipeline import PipelineModel


In [2]:
spark = SparkSession.builder\
        .appName("demo03")\
        .getOrCreate()

24/12/27 11:09:51 WARN Utils: Your hostname, nilesh-pc resolves to a loopback address: 127.0.1.1; using 192.168.1.101 instead (on interface wlp0s20f3)
24/12/27 11:09:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/27 11:09:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/27 11:09:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [11]:
gender = "Female"     # input from user
salary = 76000.0    # input from user
age = 56            # input from user
id = 0              # default values (not needed)
purchased = 0       # default values (not needed)

schema = "CustID INT,Gender STRING,Age INT,Salary DOUBLE,Purchased INT"
df = spark.createDataFrame([
    (id, gender, age, salary, purchased)
], schema=schema)

df.printSchema()
df.show()

root
 |-- CustID: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Salary: double (nullable = true)
 |-- Purchased: integer (nullable = true)

+------+------+---+-------+---------+
|CustID|Gender|Age| Salary|Purchased|
+------+------+---+-------+---------+
|     0|Female| 56|76000.0|        0|
+------+------+---+-------+---------+



In [5]:
model = PipelineModel.load("file:///tmp/model2")
print(model)

PipelineModel_aab778d0df34


In [12]:
predictions = model.transform(df)

predictions.printSchema()
predictions.show(truncate=False)

root
 |-- CustID: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Salary: double (nullable = true)
 |-- Purchased: integer (nullable = true)
 |-- GenderIndexed: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)

+------+------+---+-------+---------+-------------+------------------+----------------------------------------+-----------------------------------------+----------+
|CustID|Gender|Age|Salary |Purchased|GenderIndexed|features          |rawPrediction                           |probability                              |prediction|
+------+------+---+-------+---------+-------------+------------------+----------------------------------------+-----------------------------------------+----------+
|0     |Female|56 |76000.0|0        |0.0          |[56.0,76000.0,0.0]|[-3.231899429155465

In [13]:
result = predictions.select("prediction")\
    .first()

if result[0] == 1.0:
    print("Will Purchase")
else:
    print("Will Not Purchase")

Will Purchase
